In [ ]:
!pip install git+https://github.com/huggingface/transformers accelerate qwen-vl-utils bitsandbytes

In [ ]:
!pip install torch

In [ ]:
from datasets import Dataset

# Percorso del file salvato
# json_path = "/kaggle/input/final-test-set/final_dataset.json"
dataset_path = '/kaggle/input/summarized-dataset/dataset_summarized'

# Carica il dataset Hugging Face
# hf_dataset = Dataset.from_json(json_path)
hf_dataset = Dataset.load_from_disk(dataset_path)

# Controlla un esempio
print(hf_dataset)

In [ ]:
from transformers import pipeline

# Carica il classificatore zero-shot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Etichette da prevedere
import re

LABELS = ["real", "fake"]

def extract_label_from_response(response: str) -> int:
    result = classifier(response, LABELS)
    return LABELS.index(result["labels"][0])

In [ ]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
import os

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Few-shot esempi per real/fake
FEW_SHOT_EXAMPLES = [
    {
        "context": "A street photo taken at midday with perfectly natural shadows and realistic reflections on wet pavement.",
        "label": "[Real]",
        "motivation": "Consistent lighting and natural shadow falloff; reflections are coherent with the surfaces."
    },
    {
        "context": "An indoor scene where the textures on the walls appear overly smooth and uniform, and shadows lack realistic variation.",
        "label": "[Fake]",
        "motivation": "Uniform texture and flat shadows are typical AI generation artifacts."
    },
]

STATIC_PROMPTS = [
    "<image>\nIs this image real or fake? Choose one: [Real] / [Fake].",
    "<image>\nAnalyze and classify: [Real] / [Fake].",
    "<image>\nBased on lighting and texture, decide: [Real] / [Fake]."
]

class InternVLTester:
    def __init__(self, model_id="OpenGVLab/InternVL2_5-4B", device="cuda:0"):
        self.device = torch.device(device)
        self.model = AutoModel.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            load_in_8bit=True,
            low_cpu_mem_usage=True,
            trust_remote_code=True).eval()
        
        self.tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, use_fast=False)

    def build_prompt(self, prompt_type="dynamic", x_t=""):
        few_shot = "".join(
            f"Example:\nContext: {ex['context']}\nAnswer: {ex['label']} Motivation: {ex['motivation']}\n\n"
            for ex in FEW_SHOT_EXAMPLES
        )
        if prompt_type == "dynamic" and x_t:
            return (
                f"Based on the following image description and technical analysis, determine whether the image is real or fake. <image> \n\n"
                f"Image Description and Technical Analysis:\n{x_t.strip()}\n\n"
                "Question: Is this image real or fake? Please provide a reasoned answer based on both the description and your analysis of the image. Answer with [Real] / [Fake].\n"
                "Answer:"  # Il modello deve rispondere con [Real] o [Fake]
            )
        else:
            idx = int(prompt_type.split("_")[-1]) if "_" in prompt_type else 0
            template = STATIC_PROMPTS[idx]
            return few_shot + f"Question: {template}\nAnswer:"

    def generate(self, image_path: str, prompt: str) -> str:
        # 1) load+resize
        pixel_values = load_image(image_path, max_num=12).to(torch.bfloat16).cuda()
        generation_config = dict(max_new_tokens=1024, do_sample=True)
        question = prompt
        response = self.model.chat(self.tokenizer, pixel_values, question, generation_config)
        return response

    def extract_label(self, response: str) -> int:
        return extract_label_from_response(response)

    def test(self, example: dict, prompt_type="dynamic"):
        torch.cuda.empty_cache()
        prompt = self.build_prompt(prompt_type, example.get("summarized_x_t",""))
        resp = self.generate(example["image_k"], prompt)
        label = self.extract_label(resp)
        return label, resp

# wrapper
_tester = InternVLTester()
def test_intern(example, prompt_type="dynamic"):
    return _tester.test(example, prompt_type)


In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk

# DATASET_PATH   = "/kaggle/input/test-dataset-kaggle/test_dataset_kaggle"
CHECKPOINT_CSV = "/kaggle/working/dynamic_new_dataset_sum_InternVL.csv"
PROMPT_TYPE    = "dynamic"
CHECKPOINT_EVERY = 30  # salva ogni 30 campioni

# 1) Carica dataset
#dataset = load_from_disk(DATASET_PATH)

# 2) Se esiste già un CSV di checkpoint, riloadalo e salta i campioni già processati
if os.path.exists(CHECKPOINT_CSV):
    df = pd.read_csv(CHECKPOINT_CSV)
    processed_ids = set(df["img_id"])
else:
    df = pd.DataFrame(columns=["img_id","gt_label","pred_label","response"])
    processed_ids = set()

# 3) Loop con tqdm e checkpoint ogni N campioni
for sample in tqdm(hf_dataset, desc=f"Eval {PROMPT_TYPE}", dynamic_ncols=True):
    img_id = sample["img_id"]
    if img_id in processed_ids:
        continue

    try:
        pred_label, resp = test_intern(sample, prompt_type=PROMPT_TYPE)
    except Exception as e:
        print("Errore nella generazione")
        pred_label, resp = -1, f"[ERROR] {e}"

    # prepara la nuova riga
    row = {
        "img_id":     img_id,
        "gt_label":   sample["label"],
        "pred_label": pred_label,
        "response":   resp
    }
    # concatena in un colpo solo
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    processed_ids.add(img_id)

    # salva checkpoint
    if len(processed_ids) % CHECKPOINT_EVERY == 0:
        df.to_csv(CHECKPOINT_CSV, index=False)

# 4) Alla fine salva il CSV definitivo
df.to_csv(CHECKPOINT_CSV, index=False)
print("✅ Checkpoint salvato in:", CHECKPOINT_CSV)

# 5) Calcolo accuracy su quelli validi
valid = df["pred_label"] != -1
acc = (df.loc[valid, "gt_label"] == df.loc[valid, "pred_label"]).mean()
print(f"Accuracy {PROMPT_TYPE}: {acc:.4f}")
